In [1]:
import os
import re
import sys

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

try:
    from green_agent import tools_backend, val_wrapper
except ImportError:
    # Handle case where directory structure is nested differently
    sys.path.append(os.path.join(os.getcwd(), "agentic-planning-eval"))
    from green_agent import tools_backend, val_wrapper

DOMAIN = "blocks"
INDEX = 1


def header(title):
    print(f"\n{'=' * 80}\n {title}\n{'=' * 80}")


def run_tool(func, *args):
    """Runs a tool, times it, and prints output clearly."""
    try:
        res = func(*args)
        print(res)
        return res
    except Exception as e:
        print(f"❌ ERROR: {e}")
        return None


binary_path = val_wrapper.guess_val_binary(None)

In [2]:
print(binary_path)

./green_agent/Val-20211204.1-Linux/bin/Validate


## Stateless tools

In [3]:
# 1. Overview
overview = run_tool(tools_backend.get_task_overview_nl, DOMAIN, INDEX)

Task description:
Rearrange blocks to satisfy the goal stacking configuration.

Initial situation:
A, B, C are each on the table and clear; the hand is empty.

Goal:
A is on B and B is on C (tower top->bottom: A, B, C).


In [ ]:
# 2. List objects
objects_text = run_tool(tools_backend.list_objects_nl, DOMAIN, INDEX)

In [ ]:
# 3. Describe specific object
match = re.search(r"- ([\w-]+) \(type:", objects_text)
TARGET_BLOCK = match.group(1) if match else "A"
print(f"Target Block Selected: {TARGET_BLOCK}")

run_tool(tools_backend.describe_object_nl, DOMAIN, INDEX, TARGET_BLOCK)

In [ ]:
# 4. List actions
actions_text = run_tool(tools_backend.list_actions_nl, DOMAIN)

In [ ]:
# 5. Get specific action details
run_tool(tools_backend.describe_action, DOMAIN, "pick-up")

## Stateless Plan Submission

In [ ]:
# 6. Submit full plan text (goal not fulfilled case)
plan_text = f"pick-up {TARGET_BLOCK}"
run_tool(tools_backend.submit_plan_nl, DOMAIN, INDEX, plan_text)

In [ ]:
# 7. Submit full plan text (precs not satisfied case)
plan_text = f"pick-up {TARGET_BLOCK}\npick-up {TARGET_BLOCK}"
run_tool(tools_backend.submit_plan_nl, DOMAIN, INDEX, plan_text)

In [ ]:
# 8. Submit full plan text (correct plan!)
plan_text = "pick-up B\nstack B C\npick-up A\nstack A B"
run_tool(tools_backend.submit_plan_nl, DOMAIN, INDEX, plan_text)

## Stateful Execution

In [ ]:
# 9. Reset episode
run_tool(tools_backend.reset_episode_nl, DOMAIN, INDEX)  # remove llm access

In [ ]:
# 10. Check initial state
run_tool(tools_backend.get_state_nl)

In [ ]:
# 11. Execute action (valid)
action_cmd = f"pick-up {TARGET_BLOCK}"
run_tool(tools_backend.act_nl, action_cmd)
run_tool(tools_backend.get_state_nl)

In [ ]:
# 12. Execute action (inv, precs fail)
res = run_tool(tools_backend.act_nl, action_cmd)

In [ ]:
# 13. Get history
run_tool(tools_backend.get_history_nl) # remove llm acces

In [ ]:
# 14. Submit current episode (check goal)
run_tool(tools_backend.submit_episode_nl)

In [ ]:
# 14. Undo
run_tool(tools_backend.undo_nl, 0)  # dont show llm
run_tool(tools_backend.get_history_nl)